In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import math
import copy
import random
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
univseed = 42

random.seed(univseed)
np.random.seed(univseed)
torch.manual_seed(univseed)
torch.cuda.manual_seed(univseed)
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class alexnet(nn.Module):
  def __init__(self, output_dim):
    super().__init__()
      
    self.features = nn.Sequential(
      nn.Conv2d(1, 3, (1, 1),stride=1), # uncomment to run mnist
      nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,padding=2,bias=False),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
      nn.Conv2d(in_channels=96,out_channels=192,kernel_size=5,stride=1,padding=2,bias=False),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3,stride=2,padding=0),
      nn.Conv2d(in_channels=192,out_channels=384,kernel_size=3,stride=1,padding=1,bias=False),
      nn.ReLU(inplace=True),
      nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,stride=1,padding=1,bias=False),
      nn.ReLU(inplace=True),
      nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,stride=1,padding=1,bias=False),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=0),    
    )
    
    self.classifier = nn.Sequential(
      nn.Dropout(0.5),
      nn.Linear(256 * 6 * 6, 4096),
      nn.ReLU(inplace = True),
      nn.Dropout(0.5),
      nn.Linear(4096, 4096),
      nn.ReLU(inplace = True),
      nn.Linear(4096, output_dim),
    )

  def forward(self, x):
    x = self.features(x)
    h = x.view(x.shape[0], -1)
    x = self.classifier(h)
    return x

model_alex = alexnet(10)
print('Using device:', device)
model_alex.to(device)

In [ ]:
class VGG16(nn.Module):
  def __init__(self, n_classes=10):
    super(VGG16, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1, 3, (1, 1),stride=1), #uncomment for mnist
      nn.Conv2d(3, 64, kernel_size=3, padding=1),
      #nn.Conv2d(1, 64, kernel_size=3, padding=1), #this can also be used instead of top 2 lines
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))

    self.layer2 = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Conv2d(128, 128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))

    self.layer3 = nn.Sequential(
      nn.Conv2d(128, 256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=1, padding=0),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))

    self.layer4 = nn.Sequential(
      nn.Conv2d(256, 512, kernel_size=3, padding=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=1, padding=0),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))

    self.layer5 = nn.Sequential(
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=1, padding=0),
      nn.BatchNorm2d(512),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    
    self.classifier = nn.Sequential(
      nn.Linear(in_features=512*7*7,out_features=4096),
      nn.Dropout(p=0.2),
      nn.Linear(in_features=4096, out_features=4096),
      nn.Dropout(p=0.2),
      nn.Linear(in_features=4096, out_features=n_classes)
        )   
         
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = self.layer5(out)
      out = out.view(out.size(0), -1)
      out = self.classifier(out)
      #print(out)
      return out

model_vgg = VGG16(10)
model_vgg.to(device)

In [5]:
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.Adam(model_alex.parameters(), lr=1e-3, betas=(0.9, 0.99))
# lrt = 0.001; momt = 0;
# optimizer = optim.SGD(model.parameters(), lr=lrt,momentum=momt)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((227,227)),transforms.Normalize([122.5],[255])]) #make 224, 224 for vgg

mnist = datasets.MNIST('/tmp', download=True, transform=transform)
mnist_loader = torch.utils.data.DataLoader(dataset=mnist,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)

mnist_test = datasets.MNIST('/tmp',train=False,transform=transform)
mnist_test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)

In [26]:
# totdata = datasets.ImageFolder(root='/content/drive/MyDrive/tinyImagenet_dataset',transform=transform)
# testdata, traindata = utils.data.random_split(totdata,[int(0.15*len(totdata)),len(totdata)-int(0.15*len(totdata))])
# data_loader = utils.data.DataLoader(traindata,batch_size=4, shuffle=True)
# test_loader = utils.data.DataLoader(testdata,batch_size=4, shuffle=True)

In [8]:
def findacc(testloader,net):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
      images, labels = data
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      # if(total>50):
      #   break  
  return float(correct/total)

In [ ]:
epoch = 10
trainaccs = []
testaccs = []
losses = []
nonzerowts = []

for e in range(1):
  tot_loss = 0.0
  iv=0

  for i, data in enumerate(mnist_loader,0):
    inputs, labels = data
   # inputs, labels = inputs.cuda(), labels.cuda() # add this line if using GPU

    optimizer.zero_grad()
    outputs = model_alex(inputs) #change to model_vgg for vgg
    #print(outputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    iv+=1
    # if(i>100):
    #   break
  print('[%d, %5d] loss: %.3f' %(e + 1, i + 1, tot_loss /iv))
  losses.append(tot_loss/iv)
  trainaccs.append(findacc(mnist_loader,model_alex))
  testaccs.append(findacc(mnist_test_loader,model_alex))
  zeros = 0
  allz = 0

In [ ]:
findacc(mnist_test_loader,model_alex)

In [ ]:
# import matplotlib.pyplot as plt
# modelstr = "Alexnet"

# plt.plot(range(epoch),trainaccs,label="train accuracy")
# plt.plot(range(epoch),testaccs,label="test accuracy")
# plt.title(modelstr+  ", cross entropy,adam")
# plt.ylabel("Accuracies")
# plt.xlabel("Epochs")
# plt.legend()

In [ ]:
# plt.clf()
# plt.plot(range(len(losses)),losses)
# # plt.plot(range(epoch),testaccs,label="test accuracy")
# plt.title(modelstr+ ", cross entropy, adam")
# plt.ylabel("Loss")
# plt.xlabel("Epochs")
# # plt.legend()